In [1]:
!pip install -U langsmith

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/122.3 kB ? eta -:--:--
   --- ------------------------------------ 10.2/122.3 kB ? eta -:--:--
   ------------------------------------ --- 112.6/122.3 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 122.3/122.3 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.43
    Uninstalling langsmith-0.1.43:
      Successfully uninstalled langsmith-0.1.43


In [18]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ['LANGCHAIN_PROJECT'] = "Test_project"

In [28]:
#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

azurechatmodel = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)

In [7]:
#export LANGCHAIN_TRACING_V2=true
#export LANGCHAIN_API_KEY=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
#export LANGCHAIN_PROJECT=default

In [11]:
#from openai import OpenAI
from langsmith.wrappers import wrap_openai

#openai_client = wrap_openai(OpenAI())

In [20]:
response = azurechatmodel.invoke("How langsmith is useful in GenAI?")

In [24]:
print(response.content)

Langsmith is useful in GenAI as it helps in generating high-quality and coherent text in multiple languages. It can be used to generate content for various applications such as chatbots, virtual assistants, language translation, content generation, and more. Langsmith's ability to understand and generate text in multiple languages makes it valuable in creating personalized and localized experiences for users across different cultures and languages.


In [29]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings,
)
retriever = vector.as_retriever()

In [31]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [32]:
tools = [retriever_tool]

In [25]:
from langchain import hub

# Get the prompt to use - you can modify this! - https://smith.langchain.com/hub/hwchase17/openai-functions-agent
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [33]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(azurechatmodel, tools, prompt)

In [35]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [36]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
LangSmith can help with testing in the following ways:

1. Test Case Generation: LangSmith can generate test cases automatically based on specifications or requirements. It uses techniques like symbolic execution, constraint solving, and fuzzing to generate comprehensive test cases that cover different execution paths.

2. Test Data Generation: LangSmith can generate test data automatically for different types of inputs, such as integers, strings, dates, etc. It can also generate test data that satisfies specific constraints or properties.

3. Test Execution: LangSmith can execute test cases automatically and provide detailed reports on the test results. It can detect and report any failures or errors encountered during the test execution.

4. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine the extent to which the code or system under test has been exercised. It can identify areas of the code that have not be

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith can help with testing in the following ways:\n\n1. Test Case Generation: LangSmith can generate test cases automatically based on specifications or requirements. It uses techniques like symbolic execution, constraint solving, and fuzzing to generate comprehensive test cases that cover different execution paths.\n\n2. Test Data Generation: LangSmith can generate test data automatically for different types of inputs, such as integers, strings, dates, etc. It can also generate test data that satisfies specific constraints or properties.\n\n3. Test Execution: LangSmith can execute test cases automatically and provide detailed reports on the test results. It can detect and report any failures or errors encountered during the test execution.\n\n4. Test Coverage Analysis: LangSmith can analyze the coverage of test cases to determine the extent to which the code or system under test has been exercised. It can identify area

In [38]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a brief explaination about {topic}")
parser = StrOutputParser()
chain = prompt | azurechatmodel | parser

async for chunk in chain.astream({"topic": "blackhole"}):
    print(chunk, end="", flush=True)

A black hole is a region in space where gravity is so strong that nothing, not even light, can escape its pull. It is created when a massive star collapses under its own gravitational force, causing an extremely dense and compact object. The gravitational force of a black hole is generated by its singularity, a point of infinite density at its core. The boundary surrounding the singularity is called the event horizon, beyond which anything that enters can never escape. Black holes have a profound effect on their surroundings, distorting space and time around them. They can grow in size by absorbing matter and merging with other black holes. Although black holes are invisible, their presence can be detected through their gravitational influence on nearby objects.